In [5]:
import pandas as pd
import nltk
from nltk.tokenize import TweetTokenizer
import numpy as np
import imdb


In [15]:
# nltk.download('punkt')
# nltk.download('averaged_perceptron_tagger')
# nltk.download('maxent_ne_chunker')
# nltk.download('words')

[nltk_data] Downloading package words to
[nltk_data]     C:\Users\KIIT\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\words.zip.


True

In [56]:
# year = 2015
# df = pd.read_csv(f"datasets/dataset{year}.csv")

# entities_hosts = [ " host " , " hosting ", " hosted ", " hosts ", " cohost ", " cohosting ", " cohosts "]
# entities_nominated = [ " nominated ", " nominee ", " nominees ", " nominate ", " nominates ", " nomination "]
# entities_awards = [ " best ", " Best "]
# entities_winners = [ " win ", " won ", " winning ", " wins ", " winner ", " winners " , " goes ", " receive ", " receives "]
# entities_presenters = [ " presents ", " presenters ", " presenting ", " presenter ", " present ", " presented " ]


In [21]:
#--------------------------helper----------------------------------------------#

def get_text_with_entity(df, entities):
     text_with_entities = []
     for text in df["text"]:
          if any(entity in text for entity in entities):
               text_with_entities.append(text)
     return text_with_entities

def stem_ref(sent, ref, entities, select):
     sent = TweetTokenizer().tokenize(sent)
     # for person name only
     def stem_ref_word_for_person_name(sent, ref, entities):
          sent_stemmed = []
          for word in sent: 
               if " " + word + " " in entities: sent_stemmed.append(ref)
               elif nltk.pos_tag([word])[0][1] in ["NNP", "NN"]:
                    sent_stemmed.append(word)
          return sent_stemmed

     # for award name only
     def stem_ref_word(sent,ref, entities):
          sent_stemmed = []
          for word in sent: 
               if " " + word + " " in entities: sent_stemmed.append(ref)
               else: sent_stemmed.append(word.lower())
          return sent_stemmed

     if select: return stem_ref_word_for_person_name(sent, ref, entities)
     else: return stem_ref_word(sent, ref, entities)


def get_right_bigrams(sent, i):
     return list(nltk.bigrams(sent[:i]))
def get_left_bigrams(sent, i):
     return list(nltk.bigrams(sent[i+1:]))

def get_candidates(sents, ref, entities):
     # candidates = {}
     # for sent in sents:
     #      for chunk in nltk.ne_chunk(nltk.pos_tag(nltk.word_tokenize(sent))):
     #           if hasattr(chunk, 'label'):
     #                if chunk.label().lower() == "person":
     #                     candidates[' '.join(c[0] for c in chunk)] = candidates.get(' '.join(c[0] for c in chunk), 0) + 1

     # return sorted(candidates, key=candidates.get, reverse = True)
     # this works
     all_bigrams = []
     for sent in sents:
          sent = stem_ref(sent, ref, entities, True)
          i = sent.index(ref)
          right_bigrams = get_right_bigrams(sent, i)
          left_bigrams = get_left_bigrams(sent, i)
          if right_bigrams + left_bigrams != []:
               all_bigrams = all_bigrams + right_bigrams + left_bigrams
     return all_bigrams
     

#-------------------------------------------helper----------------------------------------#





In [22]:
def host_info(df, entities):
     text_with_hosts = get_text_with_entity(df, entities)
     all_bigrams = get_candidates(text_with_hosts, "host", entities)
     freq_dic = nltk.FreqDist(all_bigrams)
     choices = []
     Max = freq_dic[freq_dic.max()] 
     for key, value in freq_dic.items():
          if value/Max > 0.9:
               key = " ".join(list(key))
               choices.append(key)

     return choices 
     

     

#comment this cell out
# host_info(df, entities_hosts)


In [23]:
def award_info(df, entities_winners, entities_presenters, entities_awards):
     text_with_winners = get_text_with_entity(df, entities_winners)
     text_with_presenters = get_text_with_entity(df, entities_presenters)
     candidates = list()
     for text in text_with_winners:
          if any(entity in text for entity in entities_awards): #filtering "best" keyword
               text = stem_ref(text, "win", entities_winners, False)
               text = stem_ref(" ".join(text), "best", entities_awards, False)
               w = text.index('win')
               b = text.index('best')
               if b > w:
                    #check only left
                    for i in range(b+2, len(text)):
                         candidates.append(" ".join(text[b:i]))
               else:
                    #check to right of "win" from "best"
                    for i in range(b+2, w):
                         candidates.append(" ".join(text[b:i]))

     for text in text_with_presenters:
          if any(entity in text for entity in entities_awards): #filtering "best" keyword
               text = stem_ref(text, "present", entities_presenters, False)
               text = stem_ref(" ".join(text), "best", entities_awards, False)
               p = text.index('present')
               b = text.index('best')
               if b > p:
                    #check only left
                    for i in range(b+2, len(text)):
                         candidates.append(" ".join(text[b:i]))
               else:
                    #check to right of "present" from "best"
                    for i in range(b+2, p):
                         candidates.append(" ".join(text[b:i]))

     def idfreq(tf):
          idf = {}
          keys = list(tf.keys())
          for key1 in keys: 
               for key2 in keys: 
                    if key1 in key2: idf[key1] = idf.get(key1, 0) + 1

          return idf

     def tf_idf_freq(tf, idf):
          tf_idf = {}
          n = len(list(tf.keys()))
          for key in tf.keys():
               tf_idf[key] = tf[key]*(np.log((1+n)/(1+idf[key]))+1)/n

          return tf_idf 

     tf = nltk.FreqDist(candidates)
     top_26 = [award[0] for award in tf.most_common()[:26]]
     idf = idfreq(tf)
     tfidf = tf_idf_freq(tf,idf)
     awards = sorted(tfidf, key=tfidf.get, reverse = True)
     return awards[:26]
     return top_26

# award_info(df, entities_winners, entities_presenters, entities_awards)

In [59]:
def nominee_info_awards(df, entities_nominated, award):
     ia = imdb.IMDb()
     text_with_nominees = get_text_with_entity(df, entities_nominated)
     candidates = list()
     text_with_nominees_for_award = [text.lower().replace(award, "") for text in text_with_nominees if award in text.lower()]
     final_results = []
     for text in text_with_nominees_for_award:
          stem_nominee = stem_ref(text, "nominee", entities_nominated, False)
          n = stem_nominee.index("nominee")
          for i in range(n+1, len(stem_nominee)-1): #left
               candidates.append(" ".join(stem_nominee[i:]))
          for i in range(0, n-2):
               candidates.append(" ".join(stem_nominee[i:n]))

     most_common = nltk.FreqDist(candidates).most_common()
     for nominee in most_common:
          nominee = ia.search_movie(nominee[0])
          if nominee!=[] and str(nominee[0]) not in final_results: 
               final_results.append(str(nominee[0]))
               if len(final_results) == 4: return final_results

def  nominee_info_people(df, entities_nominated, award):
     ia = imdb.IMDb()
     text_with_nominees = get_text_with_entity(df, entities_nominated)
     text_with_nominees_for_award = [text.lower().replace(award, "") for text in text_with_nominees if award in text.lower()]
     all_bigrams = get_candidates(text_with_nominees_for_award, "nominee", entities_nominated)
     most_common = nltk.FreqDist(all_bigrams).most_common()
     final_results = list()
     for nominee in most_common:
          nominee = ia.search_person(" ".join(nominee[0]))
          if nominee!=[] and str(nominee[0]) not in final_results: 
               final_results.append(str(nominee[0]))
               if len(final_results) == 4: return final_results
               
     return final_results
               

def nominee_info(df, entities_nominated):
     awards_dramas = {'best motion picture - drama': 'best motion picture drama', 'best motion picture - comedy or musical': 'best motion picture comedy musical', 'best animated feature film': 'best animated feature film', 'best foreign language film': 'best foreign language film', 'best screenplay - motion picture': 'best screenplay motion picture', 'best original score - motion picture': 'best original score motion picture', 'best original song - motion picture': 'best original song motion picture', 'best television series - drama': 'best television series drama', 'best television series - comedy or musical': 'best television series comedy musical', 'best mini-series or motion picture made for television': 'best miniseries motion picture made for television'}

     awards_people = {'cecil b. demille award': 'cecil b demille award', 'best performance by an actress in a motion picture - drama': 'best performance actress motion picture drama', 'best performance by an actor in a motion picture - drama': 'best performance actor motion picture drama', 'best performance by an actress in a motion picture - comedy or musical': 'best performance actress motion picture comedy musical', 'best performance by an actor in a motion picture - comedy or musical': 'best performance actor motion picture comedy musical', 'best performance by an actress in a supporting role in a motion picture': 'best performance actress supporting role motion picture', 'best performance by an actor in a supporting role in a motion picture': 'best performance actor supporting role motion picture', 'best director - motion picture': 'best director motion picture', 'best performance by an actress in a television series - drama': 'best performance actress television series drama', 'best performance by an actor in a television series - drama': 'best performance actor television series drama', 'best performance by an actress in a television series - comedy or musical': 'best performance actress television series comedy musical', 'best performance by an actor in a television series - comedy or musical': 'best performance actor television series comedy musical', 'best performance by an actress in a mini-series or motion picture made for television': 'best performance actress miniseries motion picture made for television', 'best performance by an actor in a mini-series or motion picture made for television': 'best performance actor miniseries motion picture made for television', 'best performance by an actress in a supporting role in a series, mini-series or motion picture made for television': 'best performance actress supporting role series miniseries motion picture made for television', 'best performance by an actor in a supporting role in a series, mini-series or motion picture made for television': 'best performance actor supporting role series miniseries motion picture made for television'}

     nominees = {}
     for key, value in awards_dramas.items():
          nominees[key] = nominee_info_awards(df, entities_nominated, value)

     for key, value in awards_people.items():
          nominees[key] = nominee_info_people(df, entities_nominated, value)

     return nominees

# awards_dramas = {'best motion picture - drama': 'best motion picture drama', 'best motion picture - comedy or musical': 'best motion picture comedy musical', 'best animated feature film': 'best animated feature film', 'best foreign language film': 'best foreign language film', 'best screenplay - motion picture': 'best screenplay motion picture', 'best original score - motion picture': 'best original score motion picture', 'best original song - motion picture': 'best original song motion picture', 'best television series - drama': 'best television series drama', 'best television series - comedy or musical': 'best television series comedy musical', 'best mini-series or motion picture made for television': 'best miniseries motion picture made for television'}

# awards_people = {'cecil b. demille award': 'cecil b demille award', 'best performance by an actress in a motion picture - drama': 'best performance actress motion picture drama', 'best performance by an actor in a motion picture - drama': 'best performance actor motion picture drama', 'best performance by an actress in a motion picture - comedy or musical': 'best performance actress motion picture comedy musical', 'best performance by an actor in a motion picture - comedy or musical': 'best performance actor motion picture comedy musical', 'best performance by an actress in a supporting role in a motion picture': 'best performance actress supporting role motion picture', 'best performance by an actor in a supporting role in a motion picture': 'best performance actor supporting role motion picture', 'best director - motion picture': 'best director motion picture', 'best performance by an actress in a television series - drama': 'best performance actress television series drama', 'best performance by an actor in a television series - drama': 'best performance actor television series drama', 'best performance by an actress in a television series - comedy or musical': 'best performance actress television series comedy musical', 'best performance by an actor in a television series - comedy or musical': 'best performance actor television series comedy musical', 'best performance by an actress in a mini-series or motion picture made for television': 'best performance actress miniseries motion picture made for television', 'best performance by an actor in a mini-series or motion picture made for television': 'best performance actor miniseries motion picture made for television', 'best performance by an actress in a supporting role in a series, mini-series or motion picture made for television': 'best performance actress supporting role series miniseries motion picture made for television', 'best performance by an actor in a supporting role in a series, mini-series or motion picture made for television': 'best performance actor supporting role series miniseries motion picture made for television'}

# nominees = {}
# for key, value in awards_dramas.items():
#      # print(key.upper())
#      nominees[key] = nominee_info_awards(df, entities_nominated, value)

# for key, value in awards_people.items():
#      # print(key.upper())
#      nominees[key] = nominee_info_people(df, entities_nominated, value)

# nominees

